In [26]:
import numpy as np
from numpy.typing import ArrayLike
import torch
import os


In [14]:
# want to check the std and mean shape for KIT and compare it to one datapoint.
# Why? To avoid having to calculate the same for motion-x

KIT_PATHS = {
    'motions_path': os.path.join('./KIT-ML/new_joint_vecs'),
    "labels_path" : os.path.join('./KIT-ML/texts')
}

MOX_PATHS ={
    'motions_path': os.path.join('./Motion-X/datasets/motion_data/smplx_322/GRAB/GRAB_motion/'),
    "labels_path" : os.path.join('./Motion-X/datasets/semantic_labels/GRAB'),
    "motion_faces_path" : os.path.join('./Motion-X/datasets/body_and_face_motion/GRAB/joints')
}

def get_motionx_seq_data(seq_name):
    seq_motion_path = os.path.join(MOX_PATHS['motion_faces_path'], seq_name+".npy")
    seq_label_path = os.path.join(MOX_PATHS['labels_path'], seq_name+'.txt') 

    motion = torch.tensor(np.load(seq_motion_path)).float()
    with open(seq_label_path, 'r') as f:
        seq_labels = f.readlines()
    return motion, seq_labels

def get_kit_seq_data(seq_name:str):
    seq_motion_path = os.path.join(KIT_PATHS['motions_path'], seq_name+".npy")
    seq_label_path = os.path.join(KIT_PATHS['labels_path'], seq_name+'.txt') 

    motion = torch.tensor(np.load(seq_motion_path)).float()
    with open(seq_label_path, 'r') as f:
        seq_labels = f.readlines()
    return motion, seq_labels

def drop_shapes_from_motion_arr(motion_arr: ArrayLike) -> ArrayLike:
    if isinstance(motion_arr, torch.Tensor):
        new_motion_arr = motion_arr.numpy()
    # Slice the array to exclude 'face_shape' and 'betas'
    new_motion_arr = np.concatenate((motion_arr[:, :209], motion_arr[:, 309:312]), axis=1)
    
    return new_motion_arr


In [33]:
m_kit, l_kit = get_kit_seq_data('00001')
m_mox, l_mox = get_motionx_seq_data('s1/airplane_fly_1')

# drop shape features
m_mox = drop_shapes_from_motion_arr(m_mox)

print(f"shape kit datapoint {m_kit.shape}")
print(f"shape mox datapoint {m_mox.shape}")

shape kit datapoint torch.Size([47, 251])
shape mox datapoint (279, 212)


In [16]:
mean_kit = torch.tensor(np.load('./KIT-ML/Mean.npy')).float()
std_kit = torch.tensor(np.load('./KIT-ML/Std.npy')).float()
print(mean_kit.shape)
print(std_kit.shape)

torch.Size([251])
torch.Size([251])


In [36]:
mean_mox = np.mean(m_mox,axis=0)
std_mox = np.std(m_mox,axis=0)

print(mean_mox.shape)
print(std_mox.shape)

(212,)
(212,)


In [37]:
w = np.where(std_mox==0)
print(w)

(array([], dtype=int64),)


In [39]:
np.save('Motion-X/Mean.npy',mean_mox)
np.save('Motion-X/Std.npy',std_mox)